# Context manager
- The `with` statement is used to wrap the execution of a block with methods defined by a context manager
- The context manager handles `the entry into`, and `the exit from`, the desired runtime context for the execution of the block of code
- Typical uses of context managers include saving and restoring various kinds of global state, locking and unlocking resources, closing opened files, etc
- When used as a context manager, the file will close when the code block is finished, even if an exception occurs. 
- The context manager protocol consists of the `__enter__()` and `__exit__()` methods. At the start of the with block, `__enter__()` is invoked on the context manager object. At the end of the with block,  `__exit__()` is invoked on the context manager object
- In Python, the `contextlib` module provides `@contextmanager` decorator which `allow to build a context manager from a simple generator`, instead of creating a class & implementing the protocol (enter/exit)
- The function being decorated must return `a generator-iterator` when called
- This iterator must `yield exactly one value`, which will be bound to the targets in the with statement’s as clause, if any
- At the point where the generator yields, the block nested in the with statement is executed. The generator is then resumed after the block is exited
- If an unhandled exception occurs in the block, it is re-raised inside the generator at the point where the `yield` occurred

In [12]:
import sys
import contextlib

@contextlib.contextmanager
def looking_mirror():
    original_write = sys.stdout.write
    
    def reverse_write(text):
        return original_write(text[::-1])
    sys.stdout.write = reverse_write
    yield 'exact_one_value'
    sys.stdout.write = original_write

reverse = looking_mirror()


In [13]:
reverse

In [14]:
print(dir(reverse))

['__abstractmethods__', '__call__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_recreate_cm', 'args', 'func', 'gen', 'kwds']


In [15]:
print(reverse.gen)

<generator object looking_mirror at 0x000002927A04DD60>


In [16]:
next(reverse.gen)
print('abcdefghiklm')

mlkihgfedcba


In [17]:
print('test')

tset


In [18]:
next(reverse.gen)

StopIteration: 

In [19]:
print('test')

test


In [20]:
with looking_mirror() as what:
    print('abcdefghiklm')
print(f'what: {what}')

mlkihgfedcba
what: exact_one_value


In [21]:
print('test')

test


- If an unhandled exception occurs in the with block, it is re-raised in the generator at the point where the `yield` occurred inside `looking_mirror` function. But there is no error handling there, so the `looking_mirror` function will abort without ever restoring the original `sys.stdout.write` method, leaving the system in an invalid state
- Use a `try...except...finally` statement to trap the error (if any), or ensure that some cleanup takes place

In [22]:
@contextlib.contextmanager
def looking_mirror():
    original_write = sys.stdout.write
    
    def reverse_write(text):
        return original_write(text[::-1])
    sys.stdout.write = reverse_write
    msg = 'Safely back to normal'
    try:
        yield 'exact_one_value'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero'
    finally:
        sys.stdout.write = original_write
        print(msg)

with looking_mirror() as what:
    1/0
print('abcdefghijklm')

Please DO NOT divide by zero
abcdefghijklm


In [23]:
print('test')

test


# Class as a context manager
- The context manager protocol consists of the `__enter__()` and `__exit__()` methods. At the start of the with block, `__enter__()` is invoked on the context manager object. At the end of the with block,  `__exit__()` is invoked on the context manager object
- If `__exit__()` returns None or anything but True, any exception raised in the with block will be propagated

In [24]:
import sys
class LookingMirror():
    def __enter__(self):
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'exact_one_value'
    
    def __exit__(self, exec_type, exec_value, traceback):
        sys.stdout.write = self.original_write
        if exec_type is ZeroDivisionError:
            print('Please DO NOT divide by zero')
            return True
    def reverse_write(self, text):
        self.original_write(text[::-1])

with LookingMirror() as what:
    print('abcdefghijklm')
print(what)

mlkjihgfedcba
exact_one_value


In [25]:
print('test')

test


In [26]:
with LookingMirror() as what:
    1/0
print(what)

Please DO NOT divide by zero
exact_one_value


- `__exit__()` method tells the interpreter that it has handled the exception by returning True; in that case the interpreter suppresses the exception. On the other hand if  `__exit__()` does not explicitly return a value, the interpreter gets the usual `None`, and propagates the exception
- With `@contextmanager` the default behavior is inverted: the  `__exit__` method provided by the decorator assumes any exception sent into the generator is handled and should be suppressed. Explicitly re-raising an exception in the decorated function is required if it needs to be propagated